In [1]:
stcorpus = open('std_corpus.txt','r',encoding="utf8")
stcorpus = stcorpus.readlines()

In [2]:
print('SENTENCE : ',stcorpus[0][:-2])
print()
print('LABEL :',stcorpus[0][-2])
print()
print("LENGTH : ",len(stcorpus))

SENTENCE :  The idea of trading butterflies sounds quaint, almost Victorian. But in evocative language tailored to the delicate subject, Matthew Teague braids the perilous work of Indonesia’s butterfly hunters with reminders of his beloved wife, who died of cancer. 

LABEL : 1

LENGTH :  42


In [3]:
sentences = []
labels    = []

In [4]:
import random
random.shuffle(stcorpus)

In [5]:
for i in range(42):
    sentences.append(stcorpus[i][:-2])
    labels.append(int(stcorpus[i][-2]))

In [6]:
import tensorflow as  tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
tokenizer = Tokenizer(num_words=100,oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [8]:
word_index

{'<OOV>': 1,
 'the': 2,
 'and': 3,
 'of': 4,
 'a': 5,
 'in': 6,
 'to': 7,
 'i': 8,
 'is': 9,
 'with': 10,
 'my': 11,
 'that': 12,
 'are': 13,
 'for': 14,
 'she': 15,
 'it': 16,
 'on': 17,
 'her': 18,
 'has': 19,
 'from': 20,
 'very': 21,
 'as': 22,
 'me': 23,
 'this': 24,
 'they': 25,
 'at': 26,
 'their': 27,
 'trees': 28,
 'story': 29,
 'our': 30,
 'have': 31,
 'be': 32,
 'when': 33,
 'world': 34,
 'by': 35,
 'sun': 36,
 'us': 37,
 'can': 38,
 'them': 39,
 'but': 40,
 '”': 41,
 'all': 42,
 'was': 43,
 'there': 44,
 'national': 45,
 'also': 46,
 'go': 47,
 'an': 48,
 'most': 49,
 'much': 50,
 'so': 51,
 'like': 52,
 'many': 53,
 'animal': 54,
 'how': 55,
 'were': 56,
 'great': 57,
 'day': 58,
 'such': 59,
 'used': 60,
 'his': 61,
 'up': 62,
 'through': 63,
 'or': 64,
 'other': 65,
 'last': 66,
 'we': 67,
 'love': 68,
 'about': 69,
 'make': 70,
 'called': 71,
 'animals': 72,
 'told': 73,
 'big': 74,
 'place': 75,
 'around': 76,
 'beautiful': 77,
 'time': 78,
 'elephants': 79,
 'saw': 80

In [9]:
sequences = tokenizer.texts_to_sequences(sentences)

In [10]:
ma = len(sequences[0])
for i in range(19):
    if len(sequences[i])>ma:
        ma = len(sequences[i])
print('MaxLength : ',ma)

MaxLength :  139


In [11]:
padded = pad_sequences(sequences, maxlen=ma)

In [12]:
vocab_size = len(word_index.keys())
vocab_size

1394

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          89216     
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 163,585
Trainable params: 163,585
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
import numpy as np
labels_d = np.array(labels)

In [17]:
model.fit(x = padded,
          y = labels_d,
          epochs=20,
          verbose = 1
          )

Epoch 1/20
2/2 [==============================] - 0s 7ms/step - loss: 0.6918 - accuracy: 0.4762
Epoch 2/20
2/2 [==============================] - 0s 7ms/step - loss: 0.6741 - accuracy: 0.7381
Epoch 3/20
2/2 [==============================] - 0s 8ms/step - loss: 0.6558 - accuracy: 0.7381
Epoch 4/20
2/2 [==============================] - 0s 7ms/step - loss: 0.6324 - accuracy: 0.7381
Epoch 5/20
2/2 [==============================] - 0s 8ms/step - loss: 0.6012 - accuracy: 0.7381
Epoch 6/20
2/2 [==============================] - 0s 7ms/step - loss: 0.5670 - accuracy: 0.7381
Epoch 7/20
2/2 [==============================] - 0s 7ms/step - loss: 0.5494 - accuracy: 0.7381
Epoch 8/20
2/2 [==============================] - 0s 7ms/step - loss: 0.5360 - accuracy: 0.7381
Epoch 9/20
2/2 [==============================] - 0s 6ms/step - loss: 0.5216 - accuracy: 0.7381
Epoch 10/20
2/2 [==============================] - 0s 6ms/step - loss: 0.5075 - accuracy: 0.7381
Epoch 11/20
2/2 [======================

In [18]:
test = ["My father's name is Ram. He is the best daddy in the world. I love him very much.",
        "The sun is about a trillion years old and the reaction of gasses happening there is constantly increasing it's energy"]
ts = tokenizer.texts_to_sequences(test)
ts_padded = pad_sequences(ts, maxlen=ma)

In [19]:
result = model.predict(ts_padded)

In [20]:
result

array([[0.4831702],
       [0.8380906]], dtype=float32)

In [21]:
model.save('std.h5')